In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import pickle
import sys
from functools import cache
import seaborn as sns

import scipy.signal
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV, LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import scipy
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

def load_gefcom_data():
    df = pd.read_csv("Wind/train.csv", delim_whitespace=True)
    df["date"] = pd.to_datetime(df["date"],format = "%Y%m%d%H")
    df = df[1:]
    station = '3'
    data_1 = pd.read_csv("Wind/windforecasts_wf%s.csv"%station)
    data_1["date"] = pd.to_datetime(data_1["date"],format="%Y%m%d%H")
    data_1['hors_delta'] = pd.to_timedelta(data_1.hors, unit='hours')
    data_1['date'] = data_1['date']+ data_1['hors_delta']
    data_1 = data_1[data_1['hors']<=12].drop(columns=['hors_delta', 'hors'])[:len(df)]
    data_1 = pd.concat([data_1.reset_index(drop=True), df['wp%s'%station].reset_index(drop=True)], axis=1)
    data_1 = data_1.drop(columns=['date'])
    data_1 = (data_1-data_1.mean())/data_1.std()
    X_tr = data_1.drop(columns=['wp%s'%station, 'wd'])[:4414+1]
    Y_tr = data_1['wp%s'%station][:4414+1]
    X_te = data_1.drop(columns=['wp%s'%station, 'wd'])[8760:]
    Y_te = data_1['wp%s'%station][8760:]
    return X_tr, X_te, Y_tr, Y_te

In [6]:
# print head of loaded data
X_tr, X_te, Y_tr, Y_te = load_gefcom_data()

C:\Users\henri\AppData\Local\Temp\ipykernel_19780\3721832977.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("Wind/train.csv", delim_whitespace=True)


In [7]:
X_tr.head()

,u,v,ws
0,0.610974,-0.647299,-0.177233
1,0.460924,-0.558764,-0.443764
2,0.327244,-0.530204,-0.641513
3,0.212660,-0.541628,-0.766181
4,0.122631,-0.555908,-0.843561
